In [1]:
from helper import *
import gc
import pandas as pd
import numpy as np
import warnings
import time
warnings.filterwarnings("ignore")
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score

/home/lizihaoleo/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def reduce_mem_usage(data, verbose = True):
    start_mem = data.memory_usage().sum() / 1024**2
    if verbose:
        print('Memory usage of dataframe: {:.2f} MB'.format(start_mem))
    
    for col in data.columns:
        col_type = data[col].dtype
        
        if col_type != object:
            c_min = data[col].min()
            c_max = data[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    data[col] = data[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    data[col] = data[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    data[col] = data[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    data[col] = data[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    data[col] = data[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    data[col] = data[col].astype(np.float32)
                else:
                    data[col] = data[col].astype(np.float64)

    end_mem = data.memory_usage().sum() / 1024**2
    if verbose:
        print('Memory usage after optimization: {:.2f} MB'.format(end_mem))
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return data

In [3]:
data = reduce_mem_usage(pd.read_csv('./preprocessed_data/all_data_v4.csv'))
features = data[data['TARGET'].notnull()]
test_features = data[data['TARGET'].isnull()]
# application_train = application_train.head(1000)
# test_features = pd.read_csv('./preprocessed_data/testall_data_raw.csv')

Memory usage of dataframe: 2978.94 MB
Memory usage after optimization: 1052.89 MB
Decreased by 64.7%


In [4]:
del data
gc.collect()

0

In [5]:
# Extract the ids
train_ids = features['SK_ID_CURR']
test_ids = test_features['SK_ID_CURR']

# Extract the labels for training
labels = features['TARGET']

# Remove the ids and target
features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
test_features = test_features.drop(columns = ['SK_ID_CURR','TARGET'])

In [7]:
features = pd.get_dummies(features).astype(np.int8)

MemoryError: 

In [8]:
test_features = pd.get_dummies(test_features)

In [6]:
# One Hot Encoding

features = pd.get_dummies(features)
test_features = pd.get_dummies(test_features)

# Align the dataframes by the columns
features, test_features = features.align(test_features, join = 'inner', axis = 1)

MemoryError: 

In [ ]:
def bayes_parameter_opt_lgb(X, y, init_round=50, opt_round=150, n_folds=5, random_seed=6, n_estimators=10000):
    # prepare data
    train_data = lgb.Dataset(data=X, label=y, free_raw_data=False)
    # parameters
    def lgb_eval(learning_rate, colsample_bytree, subsample, num_leaves, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
        params = {'application':'binary','num_iterations': n_estimators, 'early_stopping_round':100, 'metric':'auc'}
        params['learning_rate'] = max(learning_rate, 0)
        params['colsample_bytree'] = max(learning_rate, 0)
        params['subsample'] = max(learning_rate, 0)
        params["num_leaves"] = int(round(num_leaves))
        params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, metrics=['auc'])
        return max(cv_result['auc-mean'])
    # range colsample_bytree
    lgbBO = BayesianOptimization(lgb_eval, {
                                            'learning_rate': (10**-5, 1),
                                            'colsample_bytree': (10**-5, 1),
                                            'subsample': (10**-5, 1),
                                            'num_leaves': (16, 45),
                                            'max_depth': (3, 100),
                                            'lambda_l1': (0, 5),
                                            'lambda_l2': (0, 5),
                                            'min_split_gain': (0.00001, 0.1),
                                            'min_child_weight': (1, 100)}, random_state=random_seed)
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    # output optimization process
    lgbBO.points_to_csv("bayes_opt_result.csv")
    # return best parameters
    return lgbBO.res['max']['max_params']

In [ ]:
%%time
opt_params = bayes_parameter_opt_lgb(X=features, y=labels)

In [ ]:
print(opt_params)

In [2]:
def lgb_model(features, test_features, encoding = 'ohe', n_folds = 5):
    
    """Train and test a light gradient boosting model using
    cross validation. 
    
    Parameters
    --------
        features (pd.DataFrame): 
            dataframe of training features to use 
            for training a model. Must include the TARGET column.
        test_features (pd.DataFrame): 
            dataframe of testing features to use
            for making predictions with the model. 
        encoding (str, default = 'ohe'): 
            method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
            n_folds (int, default = 5): number of folds to use for cross validation
        
    Return
    --------
        submission (pd.DataFrame): 
            dataframe with `SK_ID_CURR` and `TARGET` probabilities
            predicted by the model.
        feature_importances (pd.DataFrame): 
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame): 
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
    """
    
    # Extract the ids
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    labels = features['TARGET']
    
    # Remove the ids and target
    features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # List for storing categorical indices
        cat_indices = []
        
        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)
    
    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = False, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', 
                                   max_depth = 14,
                                   num_leaves = 39, 
                                   colsample_bytree = 0.9883862469770172, 
                                   lambda_l2 = .6924756038716778, 
                                   learning_rate = 0.03554673894215669, 
                                   min_child_weight = 17.081258748257188, 
                                   min_split_gain = 0.02049096698043253, 
                                   subsample = 0.31253383312702765, 
                                   lambda_l1 = 4.232324068470664, 
                                   n_jobs = -1, 
                                   random_state = 6)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

In [9]:
del features, test_features
gc.collect()

7

In [3]:
data = pd.read_csv('./preprocessed_data/all_data_v3.csv')
app_train = data[data['TARGET'].notnull()]
app_test = data[data['TARGET'].isnull()]

In [ ]:
submission, fi, metrics = lgb_model(app_train, app_test)

Training Data Shape:  (307511, 859)
Testing Data Shape:  (48744, 859)
Training until validation scores don't improve for 100 rounds.
[200]	valid's auc: 0.784454	train's auc: 0.83138
[400]	valid's auc: 0.788493	train's auc: 0.86869
Early stopping, best iteration is:
[473]	valid's auc: 0.788791	train's auc: 0.879524
Training until validation scores don't improve for 100 rounds.
[200]	valid's auc: 0.787783	train's auc: 0.831354
[400]	valid's auc: 0.791615	train's auc: 0.869272
Early stopping, best iteration is:
[382]	valid's auc: 0.79174	train's auc: 0.866416
Training until validation scores don't improve for 100 rounds.
[200]	valid's auc: 0.779003	train's auc: 0.832613
[400]	valid's auc: 0.783879	train's auc: 0.870159
[600]	valid's auc: 0.784573	train's auc: 0.897139
Early stopping, best iteration is:
[567]	valid's auc: 0.784678	train's auc: 0.893035
Training until validation scores don't improve for 100 rounds.
[200]	valid's auc: 0.785521	train's auc: 0.831266
[400]	valid's auc: 0.7889	

In [17]:
metrics

,fold,train,valid
0,0,0.884038,0.788581
1,1,0.869760,0.792762
2,2,0.846631,0.783682
3,3,0.849986,0.790675
4,4,0.852049,0.792082
5,overall,0.860493,0.789467


In [18]:
submission.to_csv('./model_performance/BO_v4.csv', index = False)